In [1]:
from data.transforms import get_tensorise_h_flip_transform
from data.constants import DEFAULT_REFLACX_LABEL_COLS
from data.paths import MIMIC_EYE_PATH
from data.datasets import ReflacxObjectDetectionDataset
from torch.utils.data import DataLoader
from data.datasets import collate_fn
from data.load import seed_worker, get_dataloader_g

dataset_params_dict = {
    "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
    # "with_clinical": model_setup.use_clinical,
    "bbox_to_mask": True,
    "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
}

train_dataset = ReflacxObjectDetectionDataset(
        **dataset_params_dict, split_str="train", transforms=get_tensorise_h_flip_transform(train=False), 
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn,
    worker_init_fn=seed_worker,
    generator=get_dataloader_g(0),
)

In [2]:
from models.components.feature_extractors import ImageFeatureExtractor
from models.components.fusors import NoActionFusor
from models.components.task_performers import ObjectDetectionWithMaskParameters, ObjectDetectionWithMaskPerformer
from models.frameworks import ExtractFusePerform
from models.backbones import get_normal_backbone
from models.setup import ModelSetup
from data.constants import DEFAULT_REFLACX_LABEL_COLS



In [3]:
from utils.init import reproducibility, clean_memory_get_device

device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CPU]


In [4]:
setup = ModelSetup()
backbone = get_normal_backbone(setup)
image_extractor = ImageFeatureExtractor(backbone)
fusor = NoActionFusor()
params = ObjectDetectionWithMaskParameters()
performer = ObjectDetectionWithMaskPerformer(
    params,
    image_extractor.backbone.out_channels,
    len(DEFAULT_REFLACX_LABEL_COLS) + 1
)
# get the backbone
model = ExtractFusePerform(
    feature_extractors={"image": image_extractor},
    fusor=fusor,
    task_performers={"object-detection": performer},
)


Using pretrained backbone. mobilenet_v3


/Users/jrhs/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jrhs/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
data = next(iter(train_dataloader))
data = train_dataset.prepare_input_from_data(data, device)


/Users/jrhs/Documents/GitHub/MIMIC-Eye-applications/data/datasets.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes_df[k] = ellipse_df[
/Users/jrhs/Documents/GitHub/MIMIC-Eye-applications/data/datasets.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes_df[k] = ellipse_df[
/Users/jrhs/Documents/GitHub/MIMIC-Eye-applications/data/datasets.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [6]:
data[0]

{'image': [tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           ...,
           [0.7686, 0.7725, 0.7451,  ..., 0.8667, 0.8627, 0.8588],
           [0.7725, 0.7725, 0.7490,  ..., 0.8706, 0.8627, 0.8627],
           [0.7725, 0.7686, 0.7529,  ..., 0.8588, 0.8471, 0.8510]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           ...,
           [0.7686, 0.7725, 0.7451,  ..., 0.8667, 0.8627, 0.8588],
           [0.7725, 0.7725, 0.7490,  ..., 0.8706, 0.8627, 0.8627],
           [0.7725, 0.7686, 0.7529,  ..., 0.8588, 0.8471, 0.8510]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
           [0

In [8]:
# next

# train epoch function
model.train()


outputs = model({"images": data[0]['image']}, targets=data[1])


AttributeError: 'str' object has no attribute 'keys'

In [ ]:
data[1]

{'object-detection': [{'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            ...,
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
   'fixation_path': '/Users/jrhs/Desktop/mimic-eye/patient_14718365/REFLACX/main_data/P300R336037/fixations.csv',
   'image_path': '/Users/jrhs/Desktop/mimic-eye/patient_14718365/CXR-JPG/s59269152/5024f775-51ab5259-a943877e-ebaa6afd-2ed9fe6d.jpg',
   'dicom_id': '5024f775-51ab5259-a943877e-ebaa6afd-2ed9fe6d',
   'iscrowd': tensor([0]),
   'area': tensor([1724940.], dtype=torch.float64),
   'image_id': tensor([1101]),
   'labels': tensor([2]),
   'boxes': tensor([[1214., 1376., 2879., 2412.]], dtype=torch.float64)},
  {'masks': tensor([], size=(0, 3056, 2544), dtype=torch.uint8),
   'fixation_path': '/Users/jrhs/Desktop/mimic-eye/patient_10726497/REFLACX/main_data/P244R767994/fixations.csv',
   '

In [ ]:
outputs['object-detection']

{'lose': {'loss_classifier': tensor(1.7499, grad_fn=<NllLossBackward0>),
  'loss_box_reg': tensor(0.0311, grad_fn=<DivBackward0>),
  'loss_mask': tensor(0.6550, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
  'loss_objectness': tensor(0.6932, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
  'loss_rpn_box_reg': tensor(0.0018, dtype=torch.float64, grad_fn=<DivBackward0>)},
 'output': [{'boxes': tensor([[7.3409e+00, 0.0000e+00, 7.5543e+02, 3.3019e+02],
           [1.0763e+02, 0.0000e+00, 6.4783e+02, 1.0578e+02],
           [4.9663e+02, 2.1077e+02, 1.0372e+03, 4.2490e+02],
           [2.5563e+02, 1.0014e+02, 5.1183e+02, 5.4053e+02],
           [2.4122e+02, 9.2514e+01, 1.3162e+03, 5.3636e+02],
           [2.5134e+02, 0.0000e+00, 5.1071e+02, 2.2873e+02],
           [5.0969e+02, 7.6357e-01, 1.0418e+03, 7.4652e+02],
           [4.2204e-01, 2.1098e+02, 2.7161e+02, 4.1949e+02],
           [6.3737e+02, 1.0084e+02, 8.9652e+02, 5.4629e+02],
           [6.6503e+00, 0.0000e+00, 5.3497e+02, 2.2